In [3]:
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import mm
from reportlab.graphics.barcode import code128
from reportlab.lib import colors
import json

def generate_prescription(json_file, output_pdf):
    # Charger les données JSON
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    c = canvas.Canvas(output_pdf, pagesize=A4)
    width, height = A4

    y = height - 50

    # === Informations médecin ===
    c.setFont("Helvetica-Bold", 14)
    c.drawString(40, y, f"Dr {data['doctor']['name']}")
    y -= 18
    c.setFont("Helvetica-BoldOblique", 12)
    c.drawString(40, y, data['doctor']['specialty'])
    y -= 16
    c.setFont("Helvetica", 11)
    c.drawString(40, y, data['doctor']['city'])
    y -= 16
    c.drawString(40, y, f"Tel : {data['doctor']['phone']}")
    y -= 16
    c.drawString(40, y, f"N° Ordre : {data['doctor']['order_number']}")

    # === Titre Ordonnance ===
    y -= 50
    c.setFont("Helvetica-Bold", 26)
    c.drawCentredString(width/2, y, "Ordonnance")

    # === Informations patient ===
    y -= 40
    c.setFont("Helvetica", 11)
    c.drawString(40, y, f"Fait le : {data['date']}")
    c.drawRightString(width - 40, y, f"Patient(e) : {data['patient']['name']}")
    y -= 16
    c.drawRightString(width - 40, y, f"Âge : {data['patient']['age']} ans")

    # === Liste des médicaments ===
    y -= 40
    for med in data['prescriptions']:
        c.setFont("Helvetica-Bold", 12)
        c.drawString(40, y, med['title'])
        c.drawRightString(width - 60, y, f"Qte: {med['qty']}")
        y -= 16
        c.setFont("Helvetica", 10)
        c.drawString(40, y, f"D.C.I ({med['dci']})")
        y -= 14
        text = c.beginText(40, y)
        text.setFont("Helvetica", 10)
        for line in med['details'].split('\n'):
            text.textLine(line)
            y -= 12
        c.drawText(text)
        y -= 12

    # === Code-barres ===
    barcode_value = data.get('barcode', '0000-000')
    barcode = code128.Code128(barcode_value, barHeight=20*mm, barWidth=0.4)
    barcode.drawOn(c, 40, 100)

    # === Numéro ordonnance & site ===
    c.setFont("Helvetica-Bold", 12)
    c.drawRightString(width - 80, 110, barcode_value)
    c.setFont("Helvetica", 9)
    c.drawRightString(width - 80, 95, "https://www.pharmnet-dz.com")

    c.showPage()
    c.save()
    print(f"✅ PDF généré : {output_pdf}")


if __name__ == "__main__":
    # Exemple JSON d'entrée
    sample = {
        "doctor": {
            "name": "Bendounan Djilali",
            "specialty": "Cardiologue",
            "city": "Alger",
            "phone": "0555265120",
            "order_number": "0012/2017"
        },
        "patient": {
            "name": "IBACHIRENE Abdelkrim",
            "age": 43
        },
        "date": "7/4/2018",
        "prescriptions": [
            {
                "title": "AMOCLAN 8:1 500MG/62,5MG COMP. B/20",
                "dci": "AMOXICILLINE TRIHYDRATÉE / ACIDE CLAVULANIQUE",
                "details": "2 CP par jour pendant 8 jours",
                "qty": 1
            },
            {
                "title": "DOLIPRANE 1G COMP. B/08",
                "dci": "PARACETAMOL",
                "details": "1 CP / jour le soir",
                "qty": 1
            }
        ],
        "barcode": "0291-672"
    }

    # Sauvegarder le JSON d’exemple
    with open("ref_files/ordonnance.json", "w", encoding="utf-8") as f:
        json.dump(sample, f, ensure_ascii=False, indent=2)

    generate_prescription("ref_files/ordonnance.json", "ref_files/newordonnance.pdf")


✅ PDF généré : ref_files/newordonnance.pdf
